1data/
ML Prediction Challenge Instructions.pdf
MLapp-yhat-challenge.Rproj
Part1.ipynb
Part1.pdf
Prediction_challenge_raw.cdb
Untitled.ipynb
Variable info.xlsx
dicionary.csv
nlsy training set.csv
prediction_in_R.R
~$Variable info.xlsx
